In [1]:
import pandas as pd
from datetime import datetime
import pandas as np

Información

- Obtener los medicamentos mas relevantes
- Intentar obtener lugar_residencia, lugar_procedencia, destino_alta y vive_solo --> No tenemos datos
- Ver como hago la columna gdiagalt --> De momento lo dejaria con los códigos, no hay una clasificación clara.


### Carga de archivos

In [2]:
df = pd.read_csv('datos/DatosIdentificativosAdministrativos.csv')
df_constantes = pd.read_csv('datos/ConstantesVitales.csv')
df_glucemias = pd.read_csv('datos/GlucemiasGlucosurias.csv')
df_accidentes = pd.read_csv('datos/AccidentesPlanta.csv')
df_dietas = pd.read_csv('datos/DietasViasAlimentacion.csv')
df_antecedentes = pd.read_csv('datos/ValoracionesAntecedentesPersonales.csv')
df_valoracion_enfermeria = pd.read_csv('datos/ValoracionesEnfermeria.csv')
#df_prescripcion_farmacos = pd.read_csv('datos/PrescripcionFarmacos.csv')
df_valoracion_medica = pd.read_csv('datos/ValoracionesMedicas.csv')

/tmp/ipykernel_4631/3987163767.py:2: DtypeWarning: Columns (16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_constantes = pd.read_csv('datos/ConstantesVitales.csv')
/tmp/ipykernel_4631/3987163767.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_glucemias = pd.read_csv('datos/GlucemiasGlucosurias.csv')


### Datos identificaticos

In [3]:
# Eliminar las columnas que no tienen información relevante
df = df.drop(columns=['gprovinc', 'dnomprov', 'gpoblaci','dpoblaci','odiagini', 'odiagalt','dnomprov','ganoadme','gnumadme','gmotingr','dmotingr','gdiaging','gdiagini', 'gserulti', 'dnomserv','dsitalta','dmotalta','itraslad','odiaging','gdiasec1', 'odiasec1', 'gdiasec2', 'odiasec2'])
df.drop_duplicates(subset=['gidenpac'], keep='first', inplace=True)


In [4]:
df

,gidenpac,itipsexo,fnacipac,gcodipos,fllegada,itipingr,ireingre,finterve,faltplan,gsitalta,gmotalta,iotrocen,gdiagalt
0,GRGT192408040,H,1924-08-04T00:00:00,24191.0,2024-04-11T02:51:39,I,N,2024-04-16T00:00:00,2024-05-08T18:36:59,2,1,S,S72.102A
1,FRMR193008300,H,1930-08-30T00:00:00,24700.0,2023-06-19T15:24:27,I,N,2023-06-20T00:00:00,2023-07-04T18:32:59,2,1,S,S72.011A
2,GRPS193212171,H,1932-12-17T00:00:00,24325.0,2021-02-01T02:22:45,I,N,2021-02-02T00:00:00,2021-02-08T10:51:59,2,1,S,S72.011A
3,SNRT193308080,H,1933-08-08T00:00:00,24006.0,2023-07-02T15:31:45,I,N,2023-07-05T00:00:00,2023-07-13T17:36:59,2,1,S,S72.142A
4,SNCN193604060,H,1936-04-06T00:00:00,24010.0,2023-08-30T11:41:21,I,N,2023-08-31T00:00:00,2023-09-07T13:34:59,2,1,S,S72.141A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,LVLN194408560,M,1944-08-16T00:00:00,24226.0,2021-04-16T18:49:38,I,N,2021-04-22T00:00:00,2021-04-29T10:53:59,2,1,S,S72.141A
1976,LLCN194412440,M,1944-12-04T00:00:00,24839.0,2024-11-15T16:42:58,I,N,NaN,2024-12-18T09:36:59,6,1,S,R26.89
1977,GRCB194808471,M,1948-08-07T00:00:00,24700.0,2021-06-14T19:35:26,I,N,NaN,2021-07-02T10:33:59,2,1,S,I69.341
1978,SRGR195205570,M,1952-05-17T00:00:00,24006.0,2024-10-15T19:34:36,I,N,NaN,2024-11-09T11:55:59,9,1,S,I69.354


In [ ]:
# Reemplazar 'H' por 0 y 'M' por 1 en la columna 'itipsexo'
df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})


# Columna ds_izq_der para saber que lado de la cadera es 
# ------------------------------------------------------------------------------------------
# Eliminar las filas donde 'gdiagalt' contiene... 
# Eliminar las filas donde 'gdiagalt' no comienza con "S72."
df = df[df['gdiagalt'].str.startswith('S72.')]


# Eliminar las filas donde 'gdiagalt' contiene los códigos especificados
df = df.drop(df[df['gdiagalt'].isin(["S72.142D", "S72.90XD", "S72.102D"])].index)

# Listas de códigos para "DERECHO" y "IZQUIERDO"
derecho_codes = ["T84.010A", "S72.91XA", "S72.101A", "T84.040A", "S72.8X1A", "M97.01XD", "T84.010A", "M97.01XA"]
izquierdo_codes = ["M97.02XA", "S72.102D", "S72.012A", "M97.02XD", "T84.011A", "T84.041A", "S42.202A", "M97.02XA"]
no_especificado = ["Z51.89","M84.459D","M84.459A"]
# Categorizar la columna
def categorize_side(value):
    value = str(value)  # Convertir el valor a string por seguridad
    if any(code in value for code in no_especificado):
        return 0
    elif any(code in value for code in izquierdo_codes):
        return 1
    elif any(code in value for code in derecho_codes):
        return 2
    else:
        return None  # Marcar para eliminar

# Crear la nueva columna 'ds_izq_der' basada en la función de categorización
df['ds_izq_der'] = df['gdiagalt'].apply(categorize_side)
df['ds_izq_der'] = pd.to_numeric(df['ds_izq_der'], errors='coerce').fillna(-999).astype(int)


# Eliminar las filas donde 'ds_izq_der' es None
#df = df.dropna(subset=['ds_izq_der'])   # Solo perdemos 1
# ------------------------------------------------------------------------------------------

# Columna turnos de trabajo
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fllegada' a formato de fecha y hora
df['fllegada'] = pd.to_datetime(df['fllegada'])

# Función para asignar el turno en función de la hora
def asignar_turno(hora):
    if 8 <= hora < 15:      #Mañana
        return 0
    elif 15 <= hora < 22:   # Tarde
        return 1
    else:                   # Noche
        return 2

# Crear una nueva columna 'ds_turno' basada en la hora de 'fllegada'
df['ds_turno'] = df['fllegada'].dt.hour.apply(asignar_turno)
# ------------------------------------------------------------------------------------------

# Columna edad
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fnacipac' a formato de fecha
df['fnacipac'] = pd.to_datetime(df['fnacipac'])

# Función para calcular la edad
def calcular_edad(fecha_nacimiento):
    hoy = datetime.now()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

# Crear la nueva columna 'ds_edad' calculando la edad para cada fila
df['ds_edad'] = df['fnacipac'].apply(calcular_edad)
# ------------------------------------------------------------------------------------------

# Columnas de días estancia, preopertorio y postoperatorio
# ------------------------------------------------------------------------------------------
# Convertir las columnas de fecha a formato datetime, sin considerar las horas
df['fllegada'] = pd.to_datetime(df['fllegada']).dt.normalize()
df['faltplan'] = pd.to_datetime(df['faltplan']).dt.normalize()
df['finterve'] = pd.to_datetime(df['finterve']).dt.normalize()

# Calcular los días de estancia total (faltplan - fllegada)
df['ds_estancia'] = (df['faltplan'] - df['fllegada']).dt.days

# Calcular los días de preoperatorio incluyendo el día de intervención
df['ds_pre_oper'] = (df['finterve'] - df['fllegada']).dt.days
df['ds_pre_oper'] = pd.to_numeric(df['ds_pre_oper'], errors='coerce').fillna(-999).astype(int)


# Calcular los días de postoperatorio incluyendo el día de intervención
df['ds_post_oper'] = (df['faltplan'] - df['finterve']).dt.days
df['ds_post_oper'] = pd.to_numeric(df['ds_post_oper'], errors='coerce').fillna(-999).astype(int)

# ------------------------------------------------------------------------------------------

# Crear la columna 'ds_vivo_alta' basada en la condición de 'gsitalta'
df['ds_vivo_alta'] = df['gsitalta'].apply(lambda x: 0 if x == 7 else 1)

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ireingre'
df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})

# Crear la columna 'ds_dia_semana_llegada'(1 para lunes, 7 para domingo)
df['ds_dia_semana_llegada'] = df['fllegada'].dt.dayofweek + 1

# Crear la columna 'ds_mes_llegada'  
df['ds_mes_llegada'] = df['fllegada'].dt.month

# Obtenemos si viven en el centro o en las afueras
codigos_centro = [24001, 24002, 24003, 24004, 24005, 24006, 24007, 24008, 24009, 24010, 24012, 24070, 24071, 24080, ]  #Códigos del centro
df['ds_centro_afueras'] = df['gcodipos'].apply(lambda x: 1 if x in codigos_centro else 0)

# Reemplazar 'I' por 0 y 'U' por 1 en la columna 'itipingr'
df['itipingr'] = df['itipingr'].replace({'I': 0, 'U': 1})

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'iotrocen'
df['iotrocen'] = df['iotrocen'].replace({'N': 0, 'S': 1})

# Eliminar pacientes que no tienen nada que ver con Fractura de cadera
# Lista de valores que quieres eliminar
valores_a_eliminar = ['FDFD192912290', 'FRGR193404470', 'GNMR193911510']

# Eliminar filas donde gidenpac tenga valores en la lista
df = df[~df['gidenpac'].isin(valores_a_eliminar)]

# Eliminar columnas que ya no necesitamos
df = df.drop(columns=['faltplan'])
df = df.drop(columns=['fnacipac'])
df = df.drop(columns=['fllegada'])
df = df.drop(columns=['finterve'])

/tmp/ipykernel_4631/1077615802.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})
/tmp/ipykernel_4631/1077615802.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ds_izq_der'] = df['gdiagalt'].apply(categorize_side)
/tmp/ipykernel_4631/1077615802.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [6]:
# DEJO LO ANTERIOR COMENTADO 
"""
#Nos quedamos solo con las fracturas de cadera. Empiezan todas por S72.
# df = df[df['gdiagalt'].str.startswith('S72', na=False)]

# Eliminar filas duplicadas basadas en la columna 'gidenpac', conservando solo la primera aparición. Pasamos de 1078 a 1047 filas
# df = df.drop_duplicates(subset='gidenpac', keep='first')

# Reemplazar 'H' por 0 y 'M' por 1 en la columna 'itipsexo'
df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})


# Columna ds_izq_der para saber que lado de la cadera es 
# ------------------------------------------------------------------------------------------
# Listas de códigos para "DERECHO" y "IZQUIERDO"
#derecho_codes = ["001", "011", "051", "061", "091", "101", "111", "131", "141", "21", "301", "341", "391", "411", "421", "451", "491", "8X1", "91X"]
#izquierdo_codes = ["002", "012", "042", "092", "102", "112", "115", "142", "22", "302", "425", "435", "452", "455", "492", "92X"]

# Categorizar la columna
#def categorize_side(value):
#    value = str(value)  # Convertir el valor a string por seguridad
#    if any(code in value for code in derecho_codes):
#        return 1
#    elif any(code in value for code in izquierdo_codes):
#        return 0
#    else:
#        return None  # Marcar para eliminar

# Crear la nueva columna 'ds_izq_der' basada en la función de categorización
#df['ds_izq_der'] = df['gdiagalt'].apply(categorize_side)

# Eliminar las filas donde 'ds_izq_der' es None
#df = df.dropna(subset=['ds_izq_der'])   # Solo perdemos 1
# ------------------------------------------------------------------------------------------

# Columna turnos de trabajo
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fllegada' a formato de fecha y hora
df['fllegada'] = pd.to_datetime(df['fllegada'])

# Función para asignar el turno en función de la hora
def asignar_turno(hora):
    if 8 <= hora < 15:      #Mañana
        return 0
    elif 15 <= hora < 22:   # Tarde
        return 1
    else:                   # Noche
        return 2

# Crear una nueva columna 'ds_turno' basada en la hora de 'fllegada'
df['ds_turno'] = df['fllegada'].dt.hour.apply(asignar_turno)
# ------------------------------------------------------------------------------------------


# Columna edad
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fnacipac' a formato de fecha
df['fnacipac'] = pd.to_datetime(df['fnacipac'])

# Función para calcular la edad
def calcular_edad(fecha_nacimiento):
    hoy = datetime.now()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

# Crear la nueva columna 'ds_edad' calculando la edad para cada fila
df['ds_edad'] = df['fnacipac'].apply(calcular_edad)
# ------------------------------------------------------------------------------------------

# Columnas de días estancia, preopertorio y postoperatorio
# ------------------------------------------------------------------------------------------
# Convertir las columnas de fecha a formato datetime, sin considerar las horas
df['fllegada'] = pd.to_datetime(df['fllegada']).dt.normalize()
df['faltplan'] = pd.to_datetime(df['faltplan']).dt.normalize()
df['finterve'] = pd.to_datetime(df['finterve']).dt.normalize()

# Calcular los días de estancia total (faltplan - fllegada)
df['ds_estancia'] = (df['faltplan'] - df['fllegada']).dt.days

# Calcular los días de preoperatorio incluyendo el día de intervención
df['ds_pre_oper'] = (df['finterve'] - df['fllegada']).dt.days

# Calcular los días de postoperatorio incluyendo el día de intervención
df['ds_post_oper'] = (df['faltplan'] - df['finterve']).dt.days
# ------------------------------------------------------------------------------------------

# Crear la columna 'ds_vivo_alta' basada en la condición de 'gsitalta'
df['ds_vivo_alta'] = df['gsitalta'].apply(lambda x: 0 if x == 7 else 1)

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ireingre'
df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})

# Crear la columna 'ds_dia_semana_llegada'(1 para lunes, 7 para domingo)
df['ds_dia_semana_llegada'] = df['fllegada'].dt.dayofweek + 1

# Crear la columna 'ds_mes_llegada'  
df['ds_mes_llegada'] = df['fllegada'].dt.month

# Eliminar filas de códigos (gdiagalt) que apenas tengamos info
#codigos_permitidos = ['S72.141A', 'S72.012A', 'S72.142A', 'S72.011A', 
#                      'S72.002A', 'S72.001A', 'S72.102A', 'S72.111A', 'S72.102A','S72.101A' 'S72.111A', 'S72.112A' ,'S72.092A', 'S72.091A' ]
# Filtrar el DataFrame para quedarse solo con las filas cuyo valor en 'gdiagalt' esté en la lista
#df = df[df['gdiagalt'].isin(codigos_permitidos)]

# Crear columa tipo_fractura. 
#replace_dict = {
#    'S72.011A': 1, # Intracapsular --> Intracapsular no desplazada
#    'S72.012A': 1,  
#    'S72.141A': 2, # Intertrocantérea --> Intracapsular desplazada
#    'S72.142A': 2, 
#    'S72.111A': 4, # Pertrocantérea
#    'S72.112A': 4, 
#    'S72.101A': 4, 
#    'S72.102A': 4, 
#    'S72.091A': 5, # Otras
#    'S72.092A': 5, 
#    'S72.001A': 5, 
#    'S72.002A': 5
#}

# Mapear los valores de la columna 'gdiagalt' al nuevo valor en 'tipo_fractura'
#df['ds_tipo_fractura'] = df['gdiagalt'].map(replace_dict)

# Obtenemos si viven en el centro o en las afueras
codigos_centro = [24001, 24002, 24003, 24004, 24005, 24006, 24007, 24008, 24009, 24010, 24012, 24070, 24071, 24080, ]  #Códigos del centro
df['ds_centro_afueras'] = df['gcodipos'].apply(lambda x: 1 if x in codigos_centro else 0)


# Eliminar columnas que ya no necesitamos
#df = df.drop(columns=['faltplan'])
#df = df.drop(columns=['fnacipac'])
#df = df.drop(columns=['fllegada'])
#df = df.drop(columns=['finterve'])

# Pasamos a enteros y los vacios los dejamos en -999
#df['gcodipos'] = df['gcodipos'].fillna(-999).astype(int)
#df['gpoblaci'] = df['gpoblaci'].fillna(-999).astype(int)
#df['ds_izq_der'] = df['ds_izq_der'].fillna(-999).astype(int)
#df['ds_pre_oper'] = df['ds_pre_oper'].fillna(-999).astype(int)
#df['ds_post_oper'] = df['ds_post_oper'].fillna(-999).astype(int)

"""

'\n#Nos quedamos solo con las fracturas de cadera. Empiezan todas por S72.\n# df = df[df[\'gdiagalt\'].str.startswith(\'S72\', na=False)]\n\n# Eliminar filas duplicadas basadas en la columna \'gidenpac\', conservando solo la primera aparición. Pasamos de 1078 a 1047 filas\n# df = df.drop_duplicates(subset=\'gidenpac\', keep=\'first\')\n\n# Reemplazar \'H\' por 0 y \'M\' por 1 en la columna \'itipsexo\'\ndf[\'itipsexo\'] = df[\'itipsexo\'].replace({\'H\': 0, \'M\': 1})\n\n\n# Columna ds_izq_der para saber que lado de la cadera es \n# ------------------------------------------------------------------------------------------\n# Listas de códigos para "DERECHO" y "IZQUIERDO"\n#derecho_codes = ["001", "011", "051", "061", "091", "101", "111", "131", "141", "21", "301", "341", "391", "411", "421", "451", "491", "8X1", "91X"]\n#izquierdo_codes = ["002", "012", "042", "092", "102", "112", "115", "142", "22", "302", "425", "435", "452", "455", "492", "92X"]\n\n# Categorizar la columna\n#def cat

In [7]:
df

,gidenpac,itipsexo,gcodipos,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,ds_turno,ds_edad,ds_estancia,ds_pre_oper,ds_post_oper,ds_vivo_alta,ds_dia_semana_llegada,ds_mes_llegada,ds_centro_afueras
0,GRGT192408040,0,24191.0,0,0,2,1,1,S72.102A,-999,2,100,27,5,22,1,4,4,0
1,FRMR193008300,0,24700.0,0,0,2,1,1,S72.011A,-999,1,94,15,1,14,1,1,6,0
2,GRPS193212171,0,24325.0,0,0,2,1,1,S72.011A,-999,2,92,7,1,6,1,1,2,0
3,SNRT193308080,0,24006.0,0,0,2,1,1,S72.142A,-999,1,91,11,3,8,1,7,7,1
4,SNCN193604060,0,24010.0,0,0,2,1,1,S72.141A,-999,0,88,8,1,7,1,3,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,SRDL194004480,1,24220.0,0,0,2,1,1,S72.011A,-999,0,84,8,1,7,1,1,8,0
1969,LVSN194012491,1,24346.0,0,0,2,1,1,S72.012A,1,2,84,9,2,7,1,3,7,0
1971,DZTS194302440,1,24122.0,0,0,2,1,1,S72.141A,-999,2,82,12,5,7,1,4,10,0
1973,LNMG194312481,1,24975.0,0,0,2,1,1,S72.011A,-999,1,81,13,4,9,1,5,6,0


### Accidentes


In [8]:
# No tiene info que podamos usar
"""
# Filtrar df_accidentes para seleccionar solo las columnas relevantes
df_accidentes = df_accidentes[['gidenpac', 'ifamilia', 'gmovilid']]

# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'
df = pd.merge(df, df_accidentes, on='gidenpac', how='left')

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'
df['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})

# Cambiar el nombre de la columna 'ifamilia' a 'vive_solo'
df = df.rename(columns={'ifamilia': 'vive_solo'})

# Renombrar 'gmovilid' como 'movilidad_accidentes'
df = df.rename(columns={'gmovilid': 'movilidad_accidentes'})

# Eliminar filas duplicadas basándose en la columna 'gidenpac'
df = df.drop_duplicates(subset='gidenpac')
# Pasar a entero y añadimos -999 en los vacios
#df['vive_solo'] = df['vive_solo'].fillna(-999).astype(int)
#df['movilidad_accidentes'] = df['movilidad_accidentes'].fillna(-999).astype(int)
"""

"\n# Filtrar df_accidentes para seleccionar solo las columnas relevantes\ndf_accidentes = df_accidentes[['gidenpac', 'ifamilia', 'gmovilid']]\n\n# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'\ndf = pd.merge(df, df_accidentes, on='gidenpac', how='left')\n\n# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'\ndf['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})\n\n# Cambiar el nombre de la columna 'ifamilia' a 'vive_solo'\ndf = df.rename(columns={'ifamilia': 'vive_solo'})\n\n# Renombrar 'gmovilid' como 'movilidad_accidentes'\ndf = df.rename(columns={'gmovilid': 'movilidad_accidentes'})\n\n# Eliminar filas duplicadas basándose en la columna 'gidenpac'\ndf = df.drop_duplicates(subset='gidenpac')\n# Pasar a entero y añadimos -999 en los vacios\n#df['vive_solo'] = df['vive_solo'].fillna(-999).astype(int)\n#df['movilidad_accidentes'] = df['movilidad_accidentes'].fillna(-999).astype(int)\n"

In [9]:
df

,gidenpac,itipsexo,gcodipos,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,ds_turno,ds_edad,ds_estancia,ds_pre_oper,ds_post_oper,ds_vivo_alta,ds_dia_semana_llegada,ds_mes_llegada,ds_centro_afueras
0,GRGT192408040,0,24191.0,0,0,2,1,1,S72.102A,-999,2,100,27,5,22,1,4,4,0
1,FRMR193008300,0,24700.0,0,0,2,1,1,S72.011A,-999,1,94,15,1,14,1,1,6,0
2,GRPS193212171,0,24325.0,0,0,2,1,1,S72.011A,-999,2,92,7,1,6,1,1,2,0
3,SNRT193308080,0,24006.0,0,0,2,1,1,S72.142A,-999,1,91,11,3,8,1,7,7,1
4,SNCN193604060,0,24010.0,0,0,2,1,1,S72.141A,-999,0,88,8,1,7,1,3,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,SRDL194004480,1,24220.0,0,0,2,1,1,S72.011A,-999,0,84,8,1,7,1,1,8,0
1969,LVSN194012491,1,24346.0,0,0,2,1,1,S72.012A,1,2,84,9,2,7,1,3,7,0
1971,DZTS194302440,1,24122.0,0,0,2,1,1,S72.141A,-999,2,82,12,5,7,1,4,10,0
1973,LNMG194312481,1,24975.0,0,0,2,1,1,S72.011A,-999,1,81,13,4,9,1,5,6,0


### Constantes vitales

In [10]:
#La mayoria están vacios.
"""
# Seleccionar solo las columnas necesarias de df_constantes y eliminar duplicados tomando el primer valor no nulo
df_constantes = df_constantes[['gidenpac', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi']].dropna(how='all')
df_constantes = df_constantes.groupby('gidenpac').first().reset_index()

# Realizar el merge con df, añadiendo las constantes vitales
df = pd.merge(df, df_constantes, on='gidenpac', how='left')

df['ntensmin'] = pd.to_numeric(df['ntensmin'], errors='coerce').fillna(-999).astype(int)
df['ntensmax'] = pd.to_numeric(df['ntensmax'], errors='coerce').fillna(-999).astype(int)
df['ntempera'] = pd.to_numeric(df['ntempera'], errors='coerce').fillna(-999).astype(float)
df['nsatuoxi'] = pd.to_numeric(df['nsatuoxi'], errors='coerce').fillna(-999).astype(int)




# Pasamos a int y añadimos -999 en los vacios. En la temperatura dejamos float
#df['ntensmin'] = df['ntensmin'].fillna(-999).astype(int)
#df['ntensmax'] = df['ntensmax'].fillna(-999).astype(int)
#df['ntempera'] = df['ntempera'].fillna(-999).astype(float)
#df['nsatuoxi'] = df['nsatuoxi'].fillna(-999).astype(int)
"""

"\n# Seleccionar solo las columnas necesarias de df_constantes y eliminar duplicados tomando el primer valor no nulo\ndf_constantes = df_constantes[['gidenpac', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi']].dropna(how='all')\ndf_constantes = df_constantes.groupby('gidenpac').first().reset_index()\n\n# Realizar el merge con df, añadiendo las constantes vitales\ndf = pd.merge(df, df_constantes, on='gidenpac', how='left')\n\ndf['ntensmin'] = pd.to_numeric(df['ntensmin'], errors='coerce').fillna(-999).astype(int)\ndf['ntensmax'] = pd.to_numeric(df['ntensmax'], errors='coerce').fillna(-999).astype(int)\ndf['ntempera'] = pd.to_numeric(df['ntempera'], errors='coerce').fillna(-999).astype(float)\ndf['nsatuoxi'] = pd.to_numeric(df['nsatuoxi'], errors='coerce').fillna(-999).astype(int)\n\n\n\n\n# Pasamos a int y añadimos -999 en los vacios. En la temperatura dejamos float\n#df['ntensmin'] = df['ntensmin'].fillna(-999).astype(int)\n#df['ntensmax'] = df['ntensmax'].fillna(-999).astype(int)\n#df

In [11]:
df

,gidenpac,itipsexo,gcodipos,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,ds_turno,ds_edad,ds_estancia,ds_pre_oper,ds_post_oper,ds_vivo_alta,ds_dia_semana_llegada,ds_mes_llegada,ds_centro_afueras
0,GRGT192408040,0,24191.0,0,0,2,1,1,S72.102A,-999,2,100,27,5,22,1,4,4,0
1,FRMR193008300,0,24700.0,0,0,2,1,1,S72.011A,-999,1,94,15,1,14,1,1,6,0
2,GRPS193212171,0,24325.0,0,0,2,1,1,S72.011A,-999,2,92,7,1,6,1,1,2,0
3,SNRT193308080,0,24006.0,0,0,2,1,1,S72.142A,-999,1,91,11,3,8,1,7,7,1
4,SNCN193604060,0,24010.0,0,0,2,1,1,S72.141A,-999,0,88,8,1,7,1,3,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,SRDL194004480,1,24220.0,0,0,2,1,1,S72.011A,-999,0,84,8,1,7,1,1,8,0
1969,LVSN194012491,1,24346.0,0,0,2,1,1,S72.012A,1,2,84,9,2,7,1,3,7,0
1971,DZTS194302440,1,24122.0,0,0,2,1,1,S72.141A,-999,2,82,12,5,7,1,4,10,0
1973,LNMG194312481,1,24975.0,0,0,2,1,1,S72.011A,-999,1,81,13,4,9,1,5,6,0


### Antecedentes


In [12]:
# Seleccionar solo las columnas necesarias de 'df_antecedentes' y crear una copia
df_ant = df_antecedentes[['gidenpac', 'dconclin', 'vbivalor']].copy()

# Crear columnas en 'df_ant' basadas en el contenido de 'dconclin'
df_ant['ds_HTA'] = df_ant["dconclin"].str.contains("HTA", na=False).astype(int)
df_ant['ds_alergia_medicamentosa'] = df_ant["dconclin"].str.contains("Alergia medicamentosa").astype(int)
df_ant['ds_alergia_alimenticia'] = df_ant["dconclin"].str.contains("Alergia alimenticia").astype(int)
df_ant['ds_diabetes'] = df_ant["dconclin"].str.contains("Diabetes Mellitus").astype(int)
df_ant['ds_otras_alergias'] = df_ant["dconclin"].str.contains("Otras alergias").astype(int)

# Actualizar valores en columnas de antecedentes usando 'vbivalor' si no es nulo
columns_to_update = ['ds_HTA', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias']
for col in columns_to_update:
    df_ant[col] = df_ant.apply(lambda x: x['vbivalor'] if pd.notnull(x['vbivalor']) and x[col] == 1 else x[col], axis=1)

# Reemplazar valores específicos ('N' -> 0, 'S' -> 1, 'A' -> 1) en todo el DataFrame
df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)

# Eliminar las columnas 'dconclin' y 'vbivalor' si ya no son necesarias
df_ant.drop(columns=['dconclin', 'vbivalor'], inplace=True)

# Eliminar duplicados en el DataFrame (si aún no se ha hecho)
df_ant = df_ant.drop_duplicates()

# Agrupar por 'gidenpac' y usar 'max' para que cualquier 1 en una columna mantenga el valor como 1
df_ant = df_ant.groupby('gidenpac', as_index=False).max()

# Realizar un merge entre df y df_ant sin sufijos
df = pd.merge(df, df_ant, on='gidenpac', how='left')

# Asignar valores de df_ant solo si son 1
for col in columns_to_update:
    df[col] = df.apply(lambda x: x[col] if pd.notnull(x[col]) and x[col] == 1 else np.nan if pd.isna(x[col]) else x[col], axis=1)


/tmp/ipykernel_4631/2776989086.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)


In [13]:
df_ant

,gidenpac,ds_HTA,ds_alergia_medicamentosa,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias
0,BCDZ196610130,0,1,0,0,0
1,BCPR193304560,0,0,0,0,0
2,BDCR193008540,0,0,0,0,0
3,BDDB193612170,0,0,0,1,0
4,BDGR193408310,1,0,0,1,0
...,...,...,...,...,...,...
1898,ZPHR193007500,0,1,0,0,0
1899,ZPSR193201490,0,0,0,0,0
1900,ZQRS193604120,1,0,0,0,0
1901,ZRRD195810130,0,0,0,0,0


### Valoracion Enfermeria

In [14]:
import pandas as pd
import numpy as np

# Seleccionar solo las columnas necesarias de 'df_valoracion_enfermeria'
df_valoracion_enfermeria = df_valoracion_enfermeria[['gidenpac', 'dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor']].copy()

# Asegurarse de que 'ncodtabu', 'nvalncon', 'ovallcon', y 'vbivalor' sean numéricos
df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']] = df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].apply(pd.to_numeric, errors='coerce')

# Definir los valores de 'dconclin' que se convertirán en nuevas columnas
patterns = {
    '_movilidad': "_Movilidad",
    'movilidad': "Movilidad",
    'lugar_residencia': "G Lugar de Residencia",
    'lugar_procedencia': "G Procedencia",
    'destino_alta': "G Destino Alta",
    'Barthel': "Resultado Indice de Barthel",
    'barthel_alta': "Resultado Indice de Barthel al Alta",
    'braden': "Resultado Escala de Braden",
    'riesgo_caida': "Resultado Escala Riesgo Caidas"
}

# Crear una expresión regular unida para filtrar las filas deseadas
regex_pattern = "|".join(patterns.values())

# Filtrar el DataFrame para conservar solo las filas que coinciden con el patrón
df_valoracion_enfermeria = df_valoracion_enfermeria[df_valoracion_enfermeria['dconclin'].str.contains(regex_pattern, na=False)]

# Iterar sobre el diccionario para crear las columnas basadas en el valor máximo entre las columnas especificadas
for new_col, pattern in patterns.items():
    # Crear la nueva columna con el valor máximo de las columnas seleccionadas donde 'dconclin' coincide con el patrón
    df_valoracion_enfermeria[new_col] = np.where(
        df_valoracion_enfermeria['dconclin'].str.contains(pattern, na=False),
        df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].max(axis=1),
        np.nan
    )

# Agrupar por 'gidenpac' y mantener el valor máximo en cada columna para eliminar duplicados
df_valoracion_enfermeria = df_valoracion_enfermeria.groupby('gidenpac', as_index=False).max()

# Eliminar las columnas no deseadas del DataFrame
df_valoracion_enfermeria = df_valoracion_enfermeria.drop(columns=['dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor', '_movilidad','barthel_alta'])

# Combinar df y df_valoracion_enfermeria por la columna 'gidenpac'
df= pd.merge(df, df_valoracion_enfermeria, on='gidenpac', how='left')

# Convertimos a enteros y los Nan los pasamos a -999
df['movilidad'] = df['movilidad'].fillna(-999).astype(int)
df['lugar_residencia'] = df['lugar_residencia'].fillna(-999).astype(int)
df['lugar_procedencia'] = df['lugar_procedencia'].fillna(-999).astype(int)
df['destino_alta'] = df['destino_alta'].fillna(-999).astype(int)
df['Barthel'] = df['Barthel'].fillna(-999).astype(int)
df['braden'] = df['braden'].fillna(-999).astype(int)
df['riesgo_caida'] = df['riesgo_caida'].fillna(-999).astype(int)


In [15]:
df_valoracion_enfermeria

,gidenpac,movilidad,lugar_residencia,lugar_procedencia,destino_alta,Barthel,braden,riesgo_caida
0,BCDZ196610130,2.0,NaN,NaN,NaN,NaN,11.0,5.0
1,BDDB193612170,3.0,NaN,NaN,NaN,10.0,13.0,4.0
2,BDGR193408310,2.0,NaN,NaN,NaN,20.0,16.0,5.0
3,BDMG193204480,3.0,NaN,NaN,NaN,45.0,16.0,5.0
4,BDVL195306230,2.0,1.0,2.0,2.0,NaN,14.0,2.0
...,...,...,...,...,...,...,...,...
669,XTPS193708580,3.0,NaN,NaN,NaN,45.0,17.0,8.0
670,YGCR192612660,2.0,NaN,NaN,NaN,15.0,15.0,5.0
671,YLLF193109610,2.0,NaN,NaN,NaN,10.0,17.0,6.0
672,ZPSR193201490,3.0,NaN,NaN,NaN,20.0,16.0,4.0


In [16]:
df

,gidenpac,itipsexo,gcodipos,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,...,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias,movilidad,lugar_residencia,lugar_procedencia,destino_alta,Barthel,braden,riesgo_caida
0,GRGT192408040,0,24191.0,0,0,2,1,1,S72.102A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
1,FRMR193008300,0,24700.0,0,0,2,1,1,S72.011A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
2,GRPS193212171,0,24325.0,0,0,2,1,1,S72.011A,-999,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
3,SNRT193308080,0,24006.0,0,0,2,1,1,S72.142A,-999,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
4,SNCN193604060,0,24010.0,0,0,2,1,1,S72.141A,-999,...,1,0,0,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,SRDL194004480,1,24220.0,0,0,2,1,1,S72.011A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
983,LVSN194012491,1,24346.0,0,0,2,1,1,S72.012A,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
984,DZTS194302440,1,24122.0,0,0,2,1,1,S72.141A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
985,LNMG194312481,1,24975.0,0,0,2,1,1,S72.011A,-999,...,0,0,0,-999,-999,-999,-999,-999,-999,-999


### Glucemias

In [17]:
# Pocos datos

# Crear un conjunto con los valores de gidenpac en df_glucemias
glucemias_pacientes = set(df_glucemias['gidenpac'].dropna())

# Crear la columna 'ds_diabetes' solo en las casillas vacías. Añadimos info a la casilla ds_diabetes
df['ds_diabetes'] = df.apply(lambda row: 1 if pd.isna(row['ds_diabetes']) and row['gidenpac'] in glucemias_pacientes 
                             else (0 if pd.isna(row['ds_diabetes']) else row['ds_diabetes']), axis=1)


### Valoraciones Medicas

In [18]:
# Definir los códigos de 'gcodconc' que queremos filtrar para los diagnosticos
codigos = [1368, 1369, 1370, 1371, 1372, 1373, 1374, 1742, 1743, 1750, 1813, 1814, 1815]

# Listado de archivos a procesar
archivos = [
    'datos/ValoracionesMedicas.csv' 
]

# Lista para almacenar los DataFrames filtrados
dataframes_filtrados = []

# Iterar sobre cada archivo
for archivo in archivos:
    # Cargar el archivo
    df_v_med = pd.read_csv(archivo)
    
    # Filtrar las filas donde 'gcodconc' esté en la lista de códigos
    df_filtrado = df_v_med[df_v_med['gcodconc'].isin(codigos)]
    
    # Seleccionar solo las columnas 'gidenpac', 'gcodconc' y 'ovallcon'
    df_filtrado = df_filtrado[['gidenpac', 'gcodconc', 'ovallcon']]
    
    # Añadir el DataFrame filtrado a la lista
    dataframes_filtrados.append(df_filtrado)

# Concatenar todos los DataFrames filtrados en uno solo
df_val_medicas_combined = pd.concat(dataframes_filtrados, ignore_index=True)


# Guardar el resultado en un archivo CSV
df_val_medicas_combined.to_csv('DF_VAL_MEDICAS_COMBINED.csv', index=False)

conteo_codigos = df_val_medicas_combined['ovallcon'].value_counts()



In [19]:
# Crear la nueva columna 'ds_HTA' para detectar 'HTA' en cualquier parte del texto
df_val_medicas_combined['ds_HTA'] = df_val_medicas_combined['ovallcon'].str.contains('HTA', case=False, na=False).astype(int)
df_val_medicas_combined['ds_ITU'] = df_val_medicas_combined['ovallcon'].str.contains('ITU', case=False, na=False).astype(int)
#df_val_medicas_combined['ds_acido_folico'] = df_val_medicas_combined['ovallcon'].str.contains('ACIDO FOLICO', case=False, na=False).astype(int)
df_val_medicas_combined['ds_anemia'] = df_val_medicas_combined['ovallcon'].str.contains('ANEMIA', case=False, na=False).astype(int)
df_val_medicas_combined['ds_vitamina_d'] = df_val_medicas_combined['ovallcon'].str.contains('VITAMINA D', case=False, na=False).astype(int)
#df_val_medicas_combined['ds_obesidad'] = df_val_medicas_combined['ovallcon'].str.contains('OBESIDAD', case=False, na=False).astype(int)
#df_val_medicas_combined['ds_osteoporosis'] = df_val_medicas_combined['ovallcon'].str.contains('OSTEOPOROSIS', case=False, na=False).astype(int)


# Insuficiencia respiratoria
# ___________________________________________________________
respiratory_terms = [
    'INSUFICIENCIA RESPIRATORIA AGUDA', 'INSUFICIENCIA RESPIRATORIA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL', 'INSUFICIENCIA RESPIRATORIA CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL', 'INSUFICIENCIA RESPIRATORIA 2RIA', 
    'INSUFICIENCIA RESPIRATORIA GLOBAL', 'INSUFICIENCIA RESPIRATORIA AGUDA SECUNDARIA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL RESUELTA', 'EPOC AGUDIZADO', 'EPOC', 'EPOC REAGUDIZADO'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(respiratory_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_respiratoria'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Diabetes
# ___________________________________________________________
diabetes_terms = [
    'DM TIPO 2 ', 'DM TIPO II', 'DIABETES MELLITUS', 'DM-II',  'DIABETES MELLITUS TIPO II','DIABETES'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(diabetes_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_diabetes'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Infección respiratoria
# ___________________________________________________________
infeccion_respiratoria_terms = [
   'INFECCIÓN RESPIRATORIA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS',  'INFECCIÓN RESPIRATORIA DE VÍAS INFERIORES', 'INFECCION RESPIRATORIA', 'INFECCION RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS NO CONDENSANTE'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(infeccion_respiratoria_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
#df_val_medicas_combined['ds_infeccion_respiratoria'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia Cardiaca
# ___________________________________________________________
insuficiencia_cardiaca_terms = [
'INSUFICIENCIA CARDIACA', 'INSUFICIENCIA CARDIACA DESCOMPENSADA',  'INSUFICIENCIA RESPIRATORIA CRÓNICA', 'INSUFICIENCIA CARDIACA CONGESTIVA',  'I. CARDIACA', 'INSUFICIENCIA CARDIACA CONGESTIVA DESCOMPENSADA', 'INSUFICIENCIA CARDÍACA', 'INSUFICIENCIA CARDIACA CRÓNICA DESCOMPENSADA',  'INSUFICIENCIA CARDIACA CONGESTIVA AGUDA', 'ICC AGUDA', 'CARDIOPATIA ISQUEMICA',  'CARDIOPATÍA ISQUÉMICA', 'ICC', 'ICC DESCOMPENSADA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_cardiaca_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_cardiaca'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Deterioro cognitivo
# ___________________________________________________________
deterioro_cognitivo_terms = [
	'DETERIORO COGNITIVO', 'DETERIORO COGNITIVO', 'DEMENCIA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(deterioro_cognitivo_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_deterioro_cognitivo'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Exitus
# ___________________________________________________________
exitus_terms = [
    'EXITUS', 'ÉXITUS']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(exitus_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_exitus'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia renal
# ___________________________________________________________
insuficiencia_renal_terms = [
    'I. RENAL', 'INSUFICIENCIA RENAL', 'ENFERMEDAD RENAL CRÓNICA', 'INSUFICIENCIA RENAL AGUDA', 'INSUFICIENCIA RENAL CRÓNICA AGUDIZADA', 'INSUFICIENCIA RENAL CRONICA', 'IRC', 'ERC REAGUDIZADA', 'ERC AGUDIZADA', 'ERC']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_renal_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_renal'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________


# Eliminamos las columnas que ya no necesitamos
df_filtrado = df_val_medicas_combined.drop(columns=['gcodconc', 'ovallcon'])

# Agrupar por 'gidenpac' y aplicar max() para cada columna
df_val_medicas_procesado = df_filtrado.groupby('gidenpac').max().reset_index()


df_val_medicas_procesado.to_csv('DF_VAL_MEDICAS_PROCESED.csv', index=False)



In [20]:
df

,gidenpac,itipsexo,gcodipos,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,...,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias,movilidad,lugar_residencia,lugar_procedencia,destino_alta,Barthel,braden,riesgo_caida
0,GRGT192408040,0,24191.0,0,0,2,1,1,S72.102A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
1,FRMR193008300,0,24700.0,0,0,2,1,1,S72.011A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
2,GRPS193212171,0,24325.0,0,0,2,1,1,S72.011A,-999,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
3,SNRT193308080,0,24006.0,0,0,2,1,1,S72.142A,-999,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
4,SNCN193604060,0,24010.0,0,0,2,1,1,S72.141A,-999,...,1,0,0,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,SRDL194004480,1,24220.0,0,0,2,1,1,S72.011A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
983,LVSN194012491,1,24346.0,0,0,2,1,1,S72.012A,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
984,DZTS194302440,1,24122.0,0,0,2,1,1,S72.141A,-999,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
985,LNMG194312481,1,24975.0,0,0,2,1,1,S72.011A,-999,...,0,0,0,-999,-999,-999,-999,-999,-999,-999


### Filtrado de los datos finales

Unir el DataFrame principal (df) con el de valoraciones medicas ya filtrado y procesado (df_val_medicas_procesado)

In [21]:
# Unir df y df_val_medicas_procesado por la columna 'gidenpac'
df = df.merge(df_val_medicas_procesado, on='gidenpac', how='left')
# Reemplazar valores NaN o nulos por -999
#df.fillna(-999, inplace=True)

# Convertimos a enteros y los Nan los pasamos a -999
"""
df['ds_vitamina_d'] = df['ds_vitamina_d'].fillna(-999).astype(int)
df['ds_obesidad'] = df['ds_obesidad'].fillna(-999).astype(int)
df['ds_osteoporosis'] = df['ds_osteoporosis'].fillna(-999).astype(int)
df['ds_insuficiencia_respiratoria'] = df['ds_insuficiencia_respiratoria'].fillna(-999).astype(int)
df['ds_diabetes_y'] = df['ds_diabetes_y'].fillna(-999).astype(int)
df['ds_infeccion_respiratoria'] = df['ds_infeccion_respiratoria'].fillna(-999).astype(int)
df['ds_insuficiencia_cardiaca'] = df['ds_insuficiencia_cardiaca'].fillna(-999).astype(int)
df['ds_deterioro_cognitivo'] = df['ds_deterioro_cognitivo'].fillna(-999).astype(int)
df['ds_insuficiencia_renal'] = df['ds_insuficiencia_renal'].fillna(-999).astype(int)
df['ds_HTA_y'] = df['ds_HTA_y'].fillna(-999).astype(int)
df['ds_ITU'] = df['ds_ITU'].fillna(-999).astype(int)
df['ds_acido_folico'] = df['ds_acido_folico'].fillna(-999).astype(int)
df['ds_anemia'] = df['ds_anemia'].fillna(-999).astype(int)
"""

"\ndf['ds_vitamina_d'] = df['ds_vitamina_d'].fillna(-999).astype(int)\ndf['ds_obesidad'] = df['ds_obesidad'].fillna(-999).astype(int)\ndf['ds_osteoporosis'] = df['ds_osteoporosis'].fillna(-999).astype(int)\ndf['ds_insuficiencia_respiratoria'] = df['ds_insuficiencia_respiratoria'].fillna(-999).astype(int)\ndf['ds_diabetes_y'] = df['ds_diabetes_y'].fillna(-999).astype(int)\ndf['ds_infeccion_respiratoria'] = df['ds_infeccion_respiratoria'].fillna(-999).astype(int)\ndf['ds_insuficiencia_cardiaca'] = df['ds_insuficiencia_cardiaca'].fillna(-999).astype(int)\ndf['ds_deterioro_cognitivo'] = df['ds_deterioro_cognitivo'].fillna(-999).astype(int)\ndf['ds_insuficiencia_renal'] = df['ds_insuficiencia_renal'].fillna(-999).astype(int)\ndf['ds_HTA_y'] = df['ds_HTA_y'].fillna(-999).astype(int)\ndf['ds_ITU'] = df['ds_ITU'].fillna(-999).astype(int)\ndf['ds_acido_folico'] = df['ds_acido_folico'].fillna(-999).astype(int)\ndf['ds_anemia'] = df['ds_anemia'].fillna(-999).astype(int)\n"

In [22]:
df

,gidenpac,itipsexo,gcodipos,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,...,ds_HTA_y,ds_ITU,ds_anemia,ds_vitamina_d,ds_insuficiencia_respiratoria,ds_diabetes_y,ds_insuficiencia_cardiaca,ds_deterioro_cognitivo,ds_exitus,ds_insuficiencia_renal
0,GRGT192408040,0,24191.0,0,0,2,1,1,S72.102A,-999,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,FRMR193008300,0,24700.0,0,0,2,1,1,S72.011A,-999,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,GRPS193212171,0,24325.0,0,0,2,1,1,S72.011A,-999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SNRT193308080,0,24006.0,0,0,2,1,1,S72.142A,-999,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,SNCN193604060,0,24010.0,0,0,2,1,1,S72.141A,-999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,SRDL194004480,1,24220.0,0,0,2,1,1,S72.011A,-999,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
983,LVSN194012491,1,24346.0,0,0,2,1,1,S72.012A,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
984,DZTS194302440,1,24122.0,0,0,2,1,1,S72.141A,-999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
985,LNMG194312481,1,24975.0,0,0,2,1,1,S72.011A,-999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Vamos a combinar las 2 columnas de HTA.
- HTA: Tenemos 2. X e Y. 
	- X viene de antecedentes
	- Y de valoraciones médicas
- Podemos combinar las 2, ya que es más probable que no pongan la info, a poner una info falsa


In [23]:
df['ds_HTA'] = df[['ds_HTA_x', 'ds_HTA_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_HTA_x', 'ds_HTA_y'], inplace=True)


Hacemos lo mismo con la diabetes


In [24]:
df['ds_diabetes'] = df[['ds_diabetes_x', 'ds_diabetes_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_diabetes_x', 'ds_diabetes_y'], inplace=True)

Eliminar columnas que no nos van a servir


In [25]:
df.drop(columns=['ds_exitus'], inplace=True)    # Tenemos la varibale vivo al alta (ds_vivo_alta)
df.drop(columns=['gcodipos'], inplace=True)  # La elimino porque tengo el codigo postal (gcodpos)

Eliminamos filas que les falten datos importantes

In [26]:
"""
df = df[df['ds_pre_oper'] != -999]  # Eliminamos filas que no tengan los dias de pre o post operacion
df = df[df['ds_post_oper'] != -999]

# Si las valoraciones médicas no tienen info sobre el paciente, lo eliminamos
columnas_a_verificar = [
    'ds_ITU', 'ds_acido_folico', 'ds_anemia', 'ds_vitamina_d', 
    'ds_obesidad', 'ds_osteoporosis', 'ds_insuficiencia_respiratoria', 
    'ds_infeccion_respiratoria', 'ds_insuficiencia_cardiaca', 
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal'
]

df = df[~df[columnas_a_verificar].isin([-999]).any(axis=1)]
"""

"\ndf = df[df['ds_pre_oper'] != -999]  # Eliminamos filas que no tengan los dias de pre o post operacion\ndf = df[df['ds_post_oper'] != -999]\n\n# Si las valoraciones médicas no tienen info sobre el paciente, lo eliminamos\ncolumnas_a_verificar = [\n    'ds_ITU', 'ds_acido_folico', 'ds_anemia', 'ds_vitamina_d', \n    'ds_obesidad', 'ds_osteoporosis', 'ds_insuficiencia_respiratoria', \n    'ds_infeccion_respiratoria', 'ds_insuficiencia_cardiaca', \n    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal'\n]\n\ndf = df[~df[columnas_a_verificar].isin([-999]).any(axis=1)]\n"

In [27]:
df

,gidenpac,itipsexo,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,ds_turno,...,riesgo_caida,ds_ITU,ds_anemia,ds_vitamina_d,ds_insuficiencia_respiratoria,ds_insuficiencia_cardiaca,ds_deterioro_cognitivo,ds_insuficiencia_renal,ds_HTA,ds_diabetes
0,GRGT192408040,0,0,0,2,1,1,S72.102A,-999,2,...,-999,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
1,FRMR193008300,0,0,0,2,1,1,S72.011A,-999,1,...,-999,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
2,GRPS193212171,0,0,0,2,1,1,S72.011A,-999,2,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SNRT193308080,0,0,0,2,1,1,S72.142A,-999,1,...,-999,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,SNCN193604060,0,0,0,2,1,1,S72.141A,-999,0,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,SRDL194004480,1,0,0,2,1,1,S72.011A,-999,0,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
983,LVSN194012491,1,0,0,2,1,1,S72.012A,1,2,...,-999,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
984,DZTS194302440,1,0,0,2,1,1,S72.141A,-999,2,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
985,LNMG194312481,1,0,0,2,1,1,S72.011A,-999,1,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#df.to_csv('DATOS_PREPROCESADOS.csv', index=False)

### Prescripción Farmacos

In [29]:
# Medicamentos prescitos
# Se podria usar? 
"""
# Seleccionar solo las columnas 'gidenpac' y 'tarasist'
df_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]
df_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]

# Combinar los dos DataFrames
df_combined = pd.concat([df_prescripcion_farmacos, df_prescripcion_farmacos], ignore_index=True)

# Agrupar por 'gidenpac' y combinar los medicamentos en una lista
df_medicamentos = df_combined.groupby('gidenpac')['tarasist'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()

# Verificar el resultado
print(df_medicamentos.head())

df_medicamentos.to_csv('medicamentos.csv')
"""

"\n# Seleccionar solo las columnas 'gidenpac' y 'tarasist'\ndf_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]\ndf_prescripcion_farmacos = df_prescripcion_farmacos[['gidenpac', 'tarasist']]\n\n# Combinar los dos DataFrames\ndf_combined = pd.concat([df_prescripcion_farmacos, df_prescripcion_farmacos], ignore_index=True)\n\n# Agrupar por 'gidenpac' y combinar los medicamentos en una lista\ndf_medicamentos = df_combined.groupby('gidenpac')['tarasist'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()\n\n# Verificar el resultado\nprint(df_medicamentos.head())\n\ndf_medicamentos.to_csv('medicamentos.csv')\n"

In [30]:
# Convertir a categorico
columnas_a_convertir = ['itipsexo','ireingre','gsitalta', 'gmotalta','gdiagalt','ds_izq_der','ds_turno','ds_vivo_alta','ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_HTA','ds_alergia_medicamentosa','ds_alergia_alimenticia','ds_diabetes','ds_otras_alergias', 'lugar_residencia','lugar_procedencia','destino_alta','ds_ITU','ds_anemia','ds_vitamina_d','ds_insuficiencia_respiratoria','ds_insuficiencia_cardiaca','ds_deterioro_cognitivo','ds_insuficiencia_renal' ]
df[columnas_a_convertir] = df[columnas_a_convertir].astype('category')

In [31]:
df.to_csv('DATOS_PREPROCESADOS.csv', index=False)

In [32]:
df

,gidenpac,itipsexo,itipingr,ireingre,gsitalta,gmotalta,iotrocen,gdiagalt,ds_izq_der,ds_turno,...,riesgo_caida,ds_ITU,ds_anemia,ds_vitamina_d,ds_insuficiencia_respiratoria,ds_insuficiencia_cardiaca,ds_deterioro_cognitivo,ds_insuficiencia_renal,ds_HTA,ds_diabetes
0,GRGT192408040,0,0,0,2,1,1,S72.102A,-999,2,...,-999,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
1,FRMR193008300,0,0,0,2,1,1,S72.011A,-999,1,...,-999,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
2,GRPS193212171,0,0,0,2,1,1,S72.011A,-999,2,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SNRT193308080,0,0,0,2,1,1,S72.142A,-999,1,...,-999,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,SNCN193604060,0,0,0,2,1,1,S72.141A,-999,0,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,SRDL194004480,1,0,0,2,1,1,S72.011A,-999,0,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
983,LVSN194012491,1,0,0,2,1,1,S72.012A,1,2,...,-999,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
984,DZTS194302440,1,0,0,2,1,1,S72.141A,-999,2,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
985,LNMG194312481,1,0,0,2,1,1,S72.011A,-999,1,...,-999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Obtener el conteo de valores únicos
conteo_valores = df['gdiagalt'].value_counts()

# Mostrar el resultado
for valor, conteo in conteo_valores.items():
    print(f"Valor: {valor}, Repeticiones: {conteo}")


Valor: S72.141A, Repeticiones: 256
Valor: S72.012A, Repeticiones: 251
Valor: S72.142A, Repeticiones: 218
Valor: S72.011A, Repeticiones: 200
Valor: S72.002A, Repeticiones: 16
Valor: S72.001A, Repeticiones: 10
Valor: S72.102A, Repeticiones: 10
Valor: S72.101A, Repeticiones: 6
Valor: S72.22XA, Repeticiones: 6
Valor: S72.091A, Repeticiones: 4
Valor: S72.112A, Repeticiones: 3
Valor: S72.142D, Repeticiones: 2
Valor: S72.8X1A, Repeticiones: 1
Valor: S72.90XD, Repeticiones: 1
Valor: S72.102D, Repeticiones: 1
Valor: S72.051A, Repeticiones: 1
Valor: S72.91XA, Repeticiones: 1


In [34]:
# Contar el número de valores únicos en cada columna
unique_counts = df.nunique()
print(unique_counts)


gidenpac                         987
itipsexo                           2
itipingr                           3
ireingre                           2
gsitalta                           9
gmotalta                           5
iotrocen                           2
gdiagalt                          17
ds_izq_der                         3
ds_turno                           3
ds_edad                           50
ds_estancia                       47
ds_pre_oper                       23
ds_post_oper                      39
ds_vivo_alta                       2
ds_dia_semana_llegada              7
ds_mes_llegada                    12
ds_centro_afueras                  2
ds_alergia_medicamentosa           2
ds_alergia_alimenticia             2
ds_otras_alergias                  2
movilidad                          5
lugar_residencia                   3
lugar_procedencia                  3
destino_alta                       3
Barthel                           23
braden                            16
r